In [1]:
import os
import pandas as pd
os.chdir("../src/")

In [2]:
import import_inpi as inpi
import transform_rcs as trcs
import export

In [5]:
inpi.import_inpi()

In [6]:
os.getcwd()
os.mkdir("data2017/")
os.chdir("data2017/")
start_url = "http://data.cquest.org/inpi_rncs/imr/stock/2017/"
inpi.find_links(start_url, list_urls=[])

http://data.cquest.org/inpi_rncs/imr/stock/2017/05/
['http://data.cquest.org/inpi_rncs/imr/stock/2017/05/']
http://data.cquest.org/inpi_rncs/imr/stock/2017/05/04/
['http://data.cquest.org/inpi_rncs/imr/stock/2017/05/', 'http://data.cquest.org/inpi_rncs/imr/stock/2017/05/04/']


In [2]:
os.chdir("../")

In [3]:
def import_all_years():
    df_rep_2017, df_pm_2017  = trcs.transform_rcs("2017")
    df_rep_2018, df_pm_2018  = trcs.transform_rcs("2018")
    df_rep_2019, df_pm_2019 = trcs.transform_rcs("2019")
    df_rep_2020, df_pm_2020  = trcs.transform_rcs("2020")
    df_rep = pd.concat([df_rep_2017, df_rep_2018, df_rep_2019, df_rep_2020])
    df_pm = pd.concat([df_pm_2017, df_pm_2018, df_pm_2019, df_pm_2020])
    print(df_rep.shape)
    print(df_pm.shape)
    return df_rep, df_pm
    

In [4]:
df_rep, df_pm = import_all_years()

/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,15,17,25,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,17,32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,17,32) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,32) have mixed types. Specify dtype option on import 

Taille des tables
134
134
taille des tables
(9504907, 48)
(4296552, 29)
Taille des tables
7
8
taille des tables
(1142, 49)
(670, 29)


/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (17,32,35,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,14,25,33) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,14,15,17,24,25,27,28,29,30,31,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (32) have mixed typ

Taille des tables
668
668
taille des tables
(729802, 48)
(289656, 29)


/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (15,17,32,37,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,17,25,26,34) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,17,25,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,17,32,34) have mixed types. Specify dtype

Taille des tables
544
544
taille des tables
(907035, 49)
(414967, 29)
(11142886, 51)
(5001845, 31)


In [7]:


export.export2minio("df_pm_all_years.csv", df_pm)

In [8]:
export.export2minio("df_rep_all_years.csv", df_rep)